In [1]:
!git clone https://github.com/openai/shap-e.git
%cd shap-e
!pip install -e .

import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 14.75 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-lc877v52/clip_26f2fd88b8bd4239957e04686ed9800d
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-lc877v52/clip_26f2fd88b8bd4239957e04686ed9800d
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [32]:
batch_size = 5
guidance_scale = 12
prompts = [
    "christmas tree",
    "a fly dragon",
    "sword ",
    "AK-47",
    "ringing iphone"
]


latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs= {"texts": prompts},
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=32,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/32 [00:00<?, ?it/s]

In [33]:
render_mode = 'nerf' # you can change this to 'stf'
size = 128 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAMrCzMGwvKeySrGhIY+sUZOsAWqqMGOhOqmWR4uRcZp7X1yUS2…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAFbSxjrQtDHKllbAPFXCM1S9M02/Mk67Mke2QCm1eDC1XVK3ME…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAANXZOM3SNcvPNsbPN8nONMbMNMTMNsTKN8LMNsDLNcHFWMPGTc…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAIeGgYeFgoaFgYaFgIWEgIWEf4ODfYOCf4OCfoOCfIOCe4KCe4…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAAPz7+/v6+/r6+vr5+vr5+fn5+fb29fHy8u3t7urq6+rq6unq6+…

In [34]:
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    mesh = decode_latent_mesh(xm, latent).tri_mesh()

    # Save as PLY
    with open(f'model_{i}.ply', 'wb') as f:
        mesh.write_ply(f)

    # Save as OBJ
    with open(f'model_{i}.obj', 'w') as f:
        mesh.write_obj(f)

    print(f"Saved model_{i}.obj and model_{i}.ply ✅")


/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


Saved model_0.obj and model_0.ply ✅
Saved model_1.obj and model_1.ply ✅
Saved model_2.obj and model_2.ply ✅
Saved model_3.obj and model_3.ply ✅
Saved model_4.obj and model_4.ply ✅


In [40]:
from google.colab import files
files.download('model_0.obj')  # download one after the other and check each one.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>